# Import tables using beautifulsoup

We use the table on the wiki page https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans to get data about Neighborhoods and location. 

In [7]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
url='https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans'
r=requests.get(url)
soup=BeautifulSoup(r.content,'html5lib')
table_contents=[]
table=soup.find('table')
no_df = pd.read_html(str(table))[0]
no_df

,Neighborhood,Longitude,Latitude
0,U.S. NAVAL BASE,-90.026093,29.946085
1,ALGIERS POINT,-90.051606,29.952462
2,WHITNEY,-90.042357,29.947200
3,AUDUBON,-90.121450,29.932994
4,OLD AURORA,-90.000000,29.924440
...,...,...,...
67,READ BLVD EAST,-89.955196,30.035514
68,WEST LAKE FOREST,-89.992104,30.027043
69,LITTLE WOODS,-89.983864,30.041904
70,FAIRGROUNDS,-90.079436,29.986219


# Create a map of New Orleans #

Lets use folium to create a map of New Orleans using the coordinates obtained from the wiki.  

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

pyqtwebengine-5.12.1 | 175 KB    | ##################################### | 100% 
seaborn-0.11.1       | 4 KB      | ##################################### | 100% 
grpcio-1.37.0        | 2.2 MB    | ##################################### | 100% 
libclang-11.1.0      | 19.2 MB   | ##################################### | 100% 
backcall-0.2.0       | 13 KB     | ##################################### | 100% 
pyqt-impl-5.12.3     | 5.9 MB    | ##################################### | 100% 
numpy-1.20.2         | 5.8 MB    | ##################################### | 100% 
zfp-0.5.5            | 190 KB    | ##################################### | 100% 
pandas-1.2.4         | 11.8 MB   | ##################################### | 100% 
libdeflate-1.7       | 67 KB     | ##################################### | 100% 
libxgboost-1.3.3     | 3.3 MB    | ##################################### | 100% 
entrypoints-0.3      | 8 KB      | ##################################### | 100% 
jeepney-0.6.0        | 32 KB

pyerfa-1.7.2         | 385 KB    | ##################################### | 100% 
multidict-5.1.0      | 67 KB     | ##################################### | 100% 
google-pasta-0.2.0   | 42 KB     | ##################################### | 100% 
numexpr-2.7.3        | 201 KB    | ##################################### | 100% 
tifffile-2021.4.8    | 130 KB    | ##################################### | 100% 
jupyterlab_pygments- | 8 KB      | ##################################### | 100% 
libstdcxx-ng-9.3.0   | 4.0 MB    | ##################################### | 100% 
snowballstemmer-2.1. | 57 KB     | ##################################### | 100% 
sip-4.19.25          | 295 KB    | ##################################### | 100% 
rsa-4.7.2            | 28 KB     | ##################################### | 100% 
pytorch-1.8.0        | 43.3 MB   | ##################################### | 100% 
parso-0.8.2          | 68 KB     | ##################################### | 100% 
tornado-6.1          | 646 K

zstd-1.4.9           | 431 KB    | ##################################### | 100% 
mkl-2020.4           | 215.6 MB  | ##################################### | 100% 
libev-4.33           | 104 KB    | ##################################### | 100% 
ca-certificates-2020 | 137 KB    | ##################################### | 100% 
async-timeout-3.0.1  | 11 KB     | ##################################### | 100% 
brunsli-0.1          | 200 KB    | ##################################### | 100% 
unixodbc-2.3.9       | 293 KB    | ##################################### | 100% 
liblapack-3.9.0      | 11 KB     | ##################################### | 100% 
libssh2-1.9.0        | 226 KB    | ##################################### | 100% 
toml-0.10.2          | 18 KB     | ##################################### | 100% 
sqlalchemy-1.4.11    | 2.2 MB    | ##################################### | 100% 
six-1.15.0           | 14 KB     | ##################################### | 100% 
typing_extensions-3. | 25 KB

done


In [8]:
map_no = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(no_df['Latitude'], no_df['Longitude'], no_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_no)  
    
map_no

# Get foursquare data #
next we use four square to get the venues for each neighborhood in New Orleans

In [10]:
no_df.loc[0, 'Neighborhood']
CLIENT_ID = 'HRGG21QDGUFUEKLIX3LNBW5GZOHGUQPDESPMWSOQ5A35YG2J' # your Foursquare ID
CLIENT_SECRET = 'HRH1CXCWTYXQRBZIPRPTDOS2OVSEI1YUU15WIME4GMFM3PTY' # your Foursquare Secret
ACCESS_TOKEN = 'QT1AHH522C4OECT1XYXWP33QXFNLDQ2PX3YG45TKMEPARTLL' # your FourSquare Access Token
VERSION = '20180604'
#get location
neighborhood_latitude = no_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = no_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = no_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
LIMIT = 150
radius = 1500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()

Latitude and longitude values of U.S. NAVAL BASE are 29.94608469, -90.02609253.


# get venues in and around New Orleans #

In [11]:

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# construct json into a data frame
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Federal City Inn & Suites,Hotel,29.947682,-90.032891
1,Behrman Stadium,Stadium,29.939423,-90.030863
2,Subway,Sandwich Place,29.950125,-90.034375
3,Dairy Queen,Ice Cream Shop,29.940747,-90.013280
4,Save-A-Lot,Grocery Store,29.938155,-90.036670


# Getting venues for each neighborhood in New Orleans #

Now we go through and do the same process for each neighborhood in the dataframe. 

In [13]:
#create a function to do this for each neighborhood in Tor
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
#call function for each neighboor hood
no_venues = getNearbyVenues(names=no_df['Neighborhood'],
                                   latitudes=no_df['Latitude'],
                                   longitudes=no_df['Longitude']
                                  )

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST. ANTHONY
LAKE TERRACE & OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


In [14]:
#check size of resulting df
print(no_venues.shape)
no_venues.head()
#print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))
no_venues.groupby('Neighborhood').count()

(4280, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALGIERS POINT,100,100,100,100,100,100
AUDUBON,94,94,94,94,94,94
B. W. COOPER,100,100,100,100,100,100
BAYOU ST. JOHN,100,100,100,100,100,100
BEHRMAN,19,19,19,19,19,19
...,...,...,...,...,...,...
VILLAGE DE LEST,1,1,1,1,1,1
WEST END,47,47,47,47,47,47
WEST LAKE FOREST,8,8,8,8,8,8


# Analyse each neighborhood #

we can now look at the different categories of veneues in each neighborhood. 

In [17]:
# one hot encoding
no_onehot = pd.get_dummies(no_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
no_onehot['Neighborhood'] = no_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [no_onehot.columns[-1]] + list(no_onehot.columns[:-1])
no_onehot = no_onehot[fixed_columns]


In [18]:
#Group venues by neighborhood
no_grouped = no_onehot.groupby('Neighborhood').mean().reset_index()
no_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,ALGIERS POINT,0.000000,0.0,0.01,0.000000,0.01,0.01,0.000000,0.010000,0.000000,...,0.000000,0.0,0.0,0.01,0.00,0.0,0.000000,0.00,0.000000,0.000000
1,AUDUBON,0.031915,0.0,0.00,0.031915,0.00,0.00,0.000000,0.010638,0.000000,...,0.021277,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.010638,0.010638
2,B. W. COOPER,0.000000,0.0,0.00,0.010000,0.00,0.00,0.000000,0.010000,0.000000,...,0.010000,0.0,0.0,0.00,0.00,0.0,0.010000,0.01,0.000000,0.000000
3,BAYOU ST. JOHN,0.000000,0.0,0.00,0.020000,0.00,0.00,0.000000,0.000000,0.020000,...,0.010000,0.0,0.0,0.02,0.01,0.0,0.000000,0.00,0.020000,0.000000
4,BEHRMAN,0.000000,0.0,0.00,0.000000,0.00,0.00,0.052632,0.000000,0.052632,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,VILLAGE DE LEST,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.000000,0.000000
68,WEST END,0.000000,0.0,0.00,0.021277,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.000000,0.000000
69,WEST LAKE FOREST,0.000000,0.0,0.00,0.125000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.000000,0.000000
70,WEST RIVERSIDE,0.066667,0.0,0.00,0.040000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.026667,0.013333


# print Neighborhood with 5 most common venues #

With so much data it may be easier to just look ath the 5 most common types of venues in each neighborhood

In [20]:
#print neighborhood with 5 most common venues
num_top_venues = 5

for hood in no_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = no_grouped[no_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----ALGIERS POINT----
                       venue  freq
0                       Café  0.07
1  Cajun / Creole Restaurant  0.06
2                        Bar  0.06
3                Coffee Shop  0.04
4                  Jazz Club  0.04


----AUDUBON----
               venue  freq
0        Coffee Shop  0.11
1               Café  0.05
2        Pizza Place  0.04
3  French Restaurant  0.03
4      Grocery Store  0.03


----B. W. COOPER----
                  venue  freq
0           Coffee Shop  0.07
1        Sandwich Place  0.06
2            Restaurant  0.06
3  Fast Food Restaurant  0.04
4        Discount Store  0.03


----BAYOU ST. JOHN----
                             venue  freq
0                    Grocery Store  0.05
1  Southern / Soul Food Restaurant  0.05
2                              Bar  0.05
3                      Music Venue  0.04
4        Cajun / Creole Restaurant  0.04


----BEHRMAN----
                  venue  freq
0  Fast Food Restaurant  0.16
1        Ice Cream Shop  0.05
2     

In [21]:
#put into pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = no_grouped['Neighborhood']

for ind in np.arange(no_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(no_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Café,Cajun / Creole Restaurant,Bar,Coffee Shop,Jazz Club,Music Venue,Candy Store,Park,Sandwich Place,Pizza Place
1,AUDUBON,Coffee Shop,Café,Pizza Place,French Restaurant,Grocery Store,Zoo Exhibit,American Restaurant,Bookstore,Cocktail Bar,Spa
2,B. W. COOPER,Coffee Shop,Sandwich Place,Restaurant,Fast Food Restaurant,Discount Store,Seafood Restaurant,Fried Chicken Joint,Train Station,Bakery,Plaza
3,BAYOU ST. JOHN,Grocery Store,Southern / Soul Food Restaurant,Bar,Music Venue,Cajun / Creole Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Café,Coffee Shop
4,BEHRMAN,Fast Food Restaurant,Ice Cream Shop,Grocery Store,Convenience Store,Sandwich Place,Pharmacy,Flea Market,Shopping Mall,Chinese Restaurant,Fried Chicken Joint


# Cluster Neighborhoods #

In [26]:
from sklearn.cluster import KMeans
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
no_df = no_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# set number of clusters
kclusters = 5

no_grouped_clustering = no_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(no_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 


array([3, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 4, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3,
       0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3,
       1, 2, 3, 0, 3, 3], dtype=int32)

In [27]:
no_df.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,3,Hotel,Ice Cream Shop,Field,American Restaurant,Grocery Store,Stadium,Café,Credit Union,Harbor / Marina,Sandwich Place
1,ALGIERS POINT,-90.051606,29.952462,3,Café,Cajun / Creole Restaurant,Bar,Coffee Shop,Jazz Club,Music Venue,Candy Store,Park,Sandwich Place,Pizza Place
2,WHITNEY,-90.042357,29.947200,3,Boat or Ferry,Bar,Coffee Shop,Grocery Store,Credit Union,Scenic Lookout,Hotel,Sandwich Place,Historic Site,Bed & Breakfast
3,AUDUBON,-90.121450,29.932994,3,Coffee Shop,Café,Pizza Place,French Restaurant,Grocery Store,Zoo Exhibit,American Restaurant,Bookstore,Cocktail Bar,Spa
4,OLD AURORA,-90.000000,29.924440,0,Sandwich Place,Fast Food Restaurant,Pizza Place,American Restaurant,Pool,Southern / Soul Food Restaurant,Clothing Store,Shopping Mall,Coffee Shop,Seafood Restaurant


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
import matplotlib.cm as cm
import matplotlib.colors as colors
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(no_df['Latitude'], no_df['Longitude'], no_df['Neighborhood'], no_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examin Clusters #

In [29]:
#cluster 1
no_df.loc[no_df['Cluster Labels'] == 0, no_df.columns[[0] + list(range(4, no_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,OLD AURORA,Sandwich Place,Fast Food Restaurant,Pizza Place,American Restaurant,Pool,Southern / Soul Food Restaurant,Clothing Store,Shopping Mall,Coffee Shop,Seafood Restaurant
5,B. W. COOPER,Coffee Shop,Sandwich Place,Restaurant,Fast Food Restaurant,Discount Store,Seafood Restaurant,Fried Chicken Joint,Train Station,Bakery,Plaza
7,BEHRMAN,Fast Food Restaurant,Ice Cream Shop,Grocery Store,Convenience Store,Sandwich Place,Pharmacy,Flea Market,Shopping Mall,Chinese Restaurant,Fried Chicken Joint
9,BROADMOOR,Coffee Shop,Pizza Place,Fast Food Restaurant,Bar,Seafood Restaurant,Discount Store,Fried Chicken Joint,Mexican Restaurant,Pet Store,Pharmacy
10,MARLYVILLE - FONTAINEBLEAU,Fried Chicken Joint,Coffee Shop,Bakery,Gas Station,Pizza Place,Grocery Store,Discount Store,Art Gallery,Bar,Seafood Restaurant
11,GERT TOWN,Fried Chicken Joint,Discount Store,Grocery Store,Pharmacy,Pub,Gym,Chinese Restaurant,Gas Station,Bakery,Sandwich Place
20,NEW AURORA - ENGLISH TURN,Music Venue,Golf Course,Tennis Court,Women's Store,Farm,Breakfast Spot,Baseball Field,Dog Run,Campground,Trail
21,TALL TIMBERS - BRECHTEL,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Gym,Fried Chicken Joint,Gas Station,Wings Joint,Pharmacy,Coffee Shop,Bank
22,FISCHER DEV,Clothing Store,Shoe Store,Discount Store,Café,Breakfast Spot,Sporting Goods Shop,Hotel,Lingerie Store,Mobile Phone Shop,Accessories Store
36,HOLLYGROVE,Fried Chicken Joint,Gym,Pharmacy,Golf Course,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Gas Station,Grocery Store


In [30]:
#cluster 2
no_df.loc[no_df['Cluster Labels'] == 1, no_df.columns[[0] + list(range(4, no_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,VIAVANT - VENETIAN ISLES,Seafood Restaurant,Optical Shop,Performing Arts Venue,Park,Paper / Office Supplies Store,Outdoors & Recreation,Other Repair Shop,Other Great Outdoors,Opera House,Pet Service


In [31]:
#cluster 3
no_df.loc[no_df['Cluster Labels'] == 2, no_df.columns[[0] + list(range(4, no_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,VILLAGE DE LEST,Pier,Zoo Exhibit,Optical Shop,Park,Paper / Office Supplies Store,Outdoors & Recreation,Other Repair Shop,Other Great Outdoors,Opera House,Peruvian Restaurant


In [32]:
#cluster 4
no_df.loc[no_df['Cluster Labels'] == 3 , no_df.columns[[0] + list(range(4, no_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,Hotel,Ice Cream Shop,Field,American Restaurant,Grocery Store,Stadium,Café,Credit Union,Harbor / Marina,Sandwich Place
1,ALGIERS POINT,Café,Cajun / Creole Restaurant,Bar,Coffee Shop,Jazz Club,Music Venue,Candy Store,Park,Sandwich Place,Pizza Place
2,WHITNEY,Boat or Ferry,Bar,Coffee Shop,Grocery Store,Credit Union,Scenic Lookout,Hotel,Sandwich Place,Historic Site,Bed & Breakfast
3,AUDUBON,Coffee Shop,Café,Pizza Place,French Restaurant,Grocery Store,Zoo Exhibit,American Restaurant,Bookstore,Cocktail Bar,Spa
6,BAYOU ST. JOHN,Grocery Store,Southern / Soul Food Restaurant,Bar,Music Venue,Cajun / Creole Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Café,Coffee Shop
8,BLACK PEARL,Bar,Coffee Shop,Sandwich Place,Café,Cajun / Creole Restaurant,Vietnamese Restaurant,Pizza Place,Zoo Exhibit,Dessert Shop,French Restaurant
12,MID-CITY,Bar,Pizza Place,Grocery Store,Breakfast Spot,Cajun / Creole Restaurant,Pub,Restaurant,Coffee Shop,Southern / Soul Food Restaurant,Smoothie Shop
13,ST. CLAUDE,Bar,Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,Discount Store,Seafood Restaurant,Art Gallery,Bakery,Sandwich Place
14,CENTRAL BUSINESS DISTRICT,Hotel,Seafood Restaurant,Cajun / Creole Restaurant,Cocktail Bar,Italian Restaurant,Hotel Bar,Music Venue,Bar,Coffee Shop,French Restaurant
15,FRENCH QUARTER,Cajun / Creole Restaurant,Bar,Jazz Club,Cocktail Bar,Gay Bar,Park,Pizza Place,Café,History Museum,Hotel


In [33]:
#cluster 5
no_df.loc[no_df['Cluster Labels'] == 4, no_df.columns[[0] + list(range(4, no_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,LAKE CATHERINE,Clothing Store,Optical Shop,Performing Arts Venue,Park,Paper / Office Supplies Store,Outdoors & Recreation,Other Repair Shop,Other Great Outdoors,Opera House,Pet Service
